In [1]:
import galprime as gp

from astropy.io import fits
from astropy.table import Table

import os

from tqdm import tqdm

In [3]:

gp.combine_outputs("gprime_sims/gprime_out_bgsub/", "gprime_sims/gprime_out_bgsub_2/",  "gprime_bgsub/")

Combining profiles in model_profiles:   4%|▍         | 1/25 [00:11<04:29, 11.21s/it]


KeyboardInterrupt: 